In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
from getpass import getpass
import psycopg2

In [2]:
df = pd.read_csv('https://tiwen-buildamonster-bucket.s3.us-east-2.amazonaws.com/cleaned_dnd_monsters.csv')
df.head()

,name,url,cr,size,ac,hp,legendary,str,dex,con,int,wis,cha,fly,swim,type
0,aarakocra,https://www.aidedd.org/dnd/monstres.php?vo=aar...,0.25,Medium,12,13,NaN,10.0,14.0,10.0,11.0,12.0,11.0,Yes,No,humanoid
1,aboleth,https://www.aidedd.org/dnd/monstres.php?vo=abo...,10.00,Large,17,135,Legendary,21.0,9.0,15.0,18.0,15.0,18.0,No,No,aberration
2,acolyte,https://www.aidedd.org/dnd/monstres.php?vo=aco...,0.25,Medium,10,9,NaN,10.0,10.0,10.0,10.0,14.0,11.0,No,No,humanoid
3,adult-black-dragon,https://www.aidedd.org/dnd/monstres.php?vo=adu...,14.00,Huge,19,195,Legendary,23.0,14.0,21.0,14.0,13.0,17.0,Yes,Yes,dragon
4,adult-blue-dragon,https://www.aidedd.org/dnd/monstres.php?vo=adu...,16.00,Huge,19,225,Legendary,25.0,10.0,23.0,16.0,15.0,19.0,Yes,No,dragon


In [3]:
from sqlalchemy import create_engine
password = getpass('Please enter password')

conf ={
    'host':'buildamonsterdb.ccg2p5gbrhzd.us-east-2.rds.amazonaws.com',
    'port':'5432',
    'database':'postgres',
    'user':"postgres",
    'password':password
}
engine = create_engine("postgresql://{user}:{password}@{host}:{port}/{database}".format(**conf))
conn = engine.connect()


df.to_sql('monster_stats', con=conn, if_exists='replace', index=False)
conn.close()

Please enter password········


In [4]:
# Import Data
password = getpass('Please enter password')
# establish connection to Amazon RDS
conn = psycopg2.connect(
    host='buildamonsterdb.ccg2p5gbrhzd.us-east-2.rds.amazonaws.com',
    port=5432,
    dbname='postgres',
    user='postgres',
    password=password
)
query = "Select * from monster_stats"
mstats_df = pd.read_sql_query(query, conn)
conn.close()
# mstats_df = pd.read_csv('cleaned_dnd_monsters.csv')
mstats_df.head(5)

Please enter password········


,name,url,cr,size,ac,hp,legendary,str,dex,con,int,wis,cha,fly,swim,type
0,aarakocra,https://www.aidedd.org/dnd/monstres.php?vo=aar...,0.25,Medium,12,13,None,10.0,14.0,10.0,11.0,12.0,11.0,Yes,No,humanoid
1,aboleth,https://www.aidedd.org/dnd/monstres.php?vo=abo...,10.00,Large,17,135,Legendary,21.0,9.0,15.0,18.0,15.0,18.0,No,No,aberration
2,acolyte,https://www.aidedd.org/dnd/monstres.php?vo=aco...,0.25,Medium,10,9,None,10.0,10.0,10.0,10.0,14.0,11.0,No,No,humanoid
3,adult-black-dragon,https://www.aidedd.org/dnd/monstres.php?vo=adu...,14.00,Huge,19,195,Legendary,23.0,14.0,21.0,14.0,13.0,17.0,Yes,Yes,dragon
4,adult-blue-dragon,https://www.aidedd.org/dnd/monstres.php?vo=adu...,16.00,Huge,19,225,Legendary,25.0,10.0,23.0,16.0,15.0,19.0,Yes,No,dragon


In [5]:
# Remove Name & URL
mstats_df = mstats_df.drop(['name','url'], axis=1)
# Change Size to numerical data (Tiny=1,Small=2,Medium=3,Large=4,Huge=5,Gargantuan=6)
mstats_df['size'] = mstats_df['size'].map( {'Tiny': 1, 'Small': 2, 'Medium':3, 'Large': 4, 'Huge':5, 'Gargantuan':6})
mstats_df['size'].astype(int)
# Generate our categorical variable lists
mstats_cat = mstats_df.dtypes[mstats_df.dtypes == "object"].index.tolist()
# Check the number of unique values in each column
mstats_df[mstats_cat].nunique()

legendary     1
fly           2
swim          2
type         15
dtype: int64

In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False, drop='first')
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(mstats_df[mstats_cat]))
# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(mstats_cat)
encode_df.head()

C:\Users\trevo\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,legendary_None,fly_Yes,swim_Yes,type_beast,type_celestial,type_construct,type_dragon,type_elemental,type_fey,type_fiend,type_giant,type_humanoid,type_monstrosity,type_ooze,type_plant,type_swarmofTinybeasts,type_undead
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Merge one-hot encoded features and drop the originals
mstats_df = mstats_df.merge(encode_df,left_index=True, right_index=True)
mstats_df = mstats_df.drop(mstats_cat,1)
mstats_df.columns

C:\Users\trevo\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


Index(['cr', 'size', 'ac', 'hp', 'str', 'dex', 'con', 'int', 'wis', 'cha',
       'legendary_None', 'fly_Yes', 'swim_Yes', 'type_beast', 'type_celestial',
       'type_construct', 'type_dragon', 'type_elemental', 'type_fey',
       'type_fiend', 'type_giant', 'type_humanoid', 'type_monstrosity',
       'type_ooze', 'type_plant', 'type_swarmofTinybeasts', 'type_undead'],
      dtype='object')

In [8]:
# Split our preprocessed data into our features and target arrays
y = mstats_df["cr"].values
X = mstats_df.drop(['cr'],axis=1).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [9]:
model = LinearRegression()
model.fit(X_train, y_train)
training_score = model.score(X_train, y_train)
testing_score = model.score(X_test, y_test)


print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.9451701155806518
Testing Score: 0.886905594177855


In [10]:
model.coef_

array([ 0.07909636,  0.18098322,  0.03810668, -0.05702636, -0.03802873,
        0.12440479,  0.14611386,  0.14004083,  0.05820986, -3.02979347,
       -0.32659132, -0.43662237,  1.20509042, -0.24721319,  2.58533629,
       -0.32959655,  0.47856361, -1.49799827,  1.71708598,  0.90638153,
       -0.0469929 ,  0.782792  ,  1.6085642 ,  0.92353568,  1.68812587,
        1.27772706])

In [11]:
mstats_df.columns

Index(['cr', 'size', 'ac', 'hp', 'str', 'dex', 'con', 'int', 'wis', 'cha',
       'legendary_None', 'fly_Yes', 'swim_Yes', 'type_beast', 'type_celestial',
       'type_construct', 'type_dragon', 'type_elemental', 'type_fey',
       'type_fiend', 'type_giant', 'type_humanoid', 'type_monstrosity',
       'type_ooze', 'type_plant', 'type_swarmofTinybeasts', 'type_undead'],
      dtype='object')

In [12]:
model.intercept_

-3.088557975029013